In [1]:
import os
import sys
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *
from matplotlib.pyplot import MultipleLocator
from jinja2 import Template

In [2]:
file_path= '/Users/yantingting/Seafile/风控/模型/10 印尼/202004 老客决策树/01 Data'

# 数据拼接

In [10]:
y = load_data_from_pickle(file_path,'y.pkl')
# y.head(2)
# y['loan_id'].nunique()

#基本信息
baseinfo = load_data_from_pickle(file_path,'baseinfo.pkl')
baseinfo.drop_duplicates('loan_id','first',inplace = True)
dummy_list = ['device_approval'	,
              'product_id',
              'gender',
              'province',
              'industry_involved',
              'marital_status',
              'education']
frame1 = baseinfo[['loan_id']].copy()
for cols in dummy_list:
    temp_list = baseinfo[cols].value_counts().index.tolist()
    temp_dummy = pd.get_dummies(baseinfo[cols])[temp_list]
    temp_dummy.columns = [cols + '_' + str(col) for col in temp_dummy.columns]
    frame1 = frame1.join(temp_dummy)
baseinfo_var = baseinfo.merge(frame1,on ='loan_id')
baseinfo_var = baseinfo_var.drop(dummy_list,axis = 1)
# baseinfo_var.head()
var_dict_base = dp.VarDict(baseinfo_var.drop('loan_id', axis = 1), data_sorce='baseinfo')
var_dict_base.to_csv(os.path.join(file_path, 'var_dict_base.csv'), index = False)
var_dict_base.shape
save_data_to_pickle(baseinfo_var,file_path,'base_var.pkl')

# 历史行为
history1 = load_data_from_pickle(file_path,'history1.pkl')
history1.iloc[:,1:] = history1.iloc[:,1:].apply(pd.to_numeric)
history2 = load_data_from_pickle(file_path,'history2.pkl')
history3 = load_data_from_pickle(file_path,'history3.pkl')
history3.iloc[:,1:] = history3.iloc[:,1:].apply(pd.to_numeric)
history_var = history1.merge(history2,on = 'loan_id',how='left')\
                  .merge(history3,on = 'loan_id',how='left')
# history_var.head()
history_var.shape
var_dict_history = dp.VarDict(history_var.drop('loan_id', axis = 1), data_sorce='history')
var_dict_history.to_csv(os.path.join(file_path, 'var_dict_history.csv'), index = False)
var_dict_history.shape
save_data_to_pickle(history_var,file_path,'history_var.pkl')

# izi
izi = load_data_from_pickle(file_path,'izi.pkl')
izi_var = izi.drop('rank1',axis = 1)
# izi_var.fillna(-1,inplace = True)
var_dict_izi = dp.VarDict(izi_var.drop('loan_id', axis = 1), data_sorce='izi')
var_dict_izi.to_csv(os.path.join(file_path, 'var_dict_izi.csv'), index = False)
var_dict_izi.shape
save_data_to_pickle(izi_var,file_path,'izi_var.pkl')

# APP变量
# dummy
app_var = load_data_from_pickle(file_path,'var_app.pkl')
app_var = app_var.reset_index()
var_dict_app = dp.VarDict(app_var.drop('loan_id', axis = 1), data_sorce='app')
var_dict_app.to_csv(os.path.join(file_path, 'var_dict_app.csv'), index = False)
var_dict_app.shape
# app_var.head()
# var_app_freq
var_app_freq = pd.read_csv(file_path + 'var_app_freq.csv')
var_dict_app_freq = dp.VarDict(var_app_freq.drop('loan_id', axis = 1), data_sorce='app_freq')
var_dict_app_freq.to_csv(os.path.join(file_path, 'var_dict_app_freq.csv'), index = False)
# var_app_freq.head()
# var_app_tfidf
var_app_tfidf = pd.read_csv(file_path + 'var_app_tfidf.csv')
name1 = var_app_tfidf.columns.tolist()[1:]
name2 = [i + '_tfidf' for i in var_app_tfidf.columns.tolist()[1:] ]
var_app_tfidf.rename(columns = (dict(zip(name1,name2))),inplace = True)
var_dict_app_tfidf = dp.VarDict(var_app_tfidf.drop('loan_id', axis = 1), data_sorce='app_tfidf')
var_dict_app_tfidf.to_csv(os.path.join(file_path, 'var_dict_app_tfidf.csv'), index = False)
# var_app_tfidf.head()

var_dict = pd.concat([var_dict_base, var_dict_history, var_dict_izi,var_dict_app,var_dict_app_freq,var_dict_app_tfidf],axis = 0)
var_dict.to_excel(os.path.join(file_path,'dict_all.xlsx'), index = False)

df_all = y.merge(baseinfo_var,on = 'loan_id',how='left')\
           .merge(history_var,on = 'loan_id',how='left')\
           .merge(izi_var,on = 'loan_id',how='left')\
           .merge(app_var,on = 'loan_id',how='left')\
           .merge(var_app_freq,on = 'loan_id',how='left')\
           .merge(var_app_tfidf,on = 'loan_id',how='left')
save_data_to_pickle(df_all,file_path,'df_all.pkl')
df_all.shape
df_all.head()

# EDA
# eda = ss.eda(df_all,file_path)
# eda.head()

(77, 5)

(41107, 167)

(166, 5)

(27, 5)

(222, 5)

(41107, 693)

,loan_id,effective_date,due_date,approved_period,late_day,dpd1,age,monthly_salary,device_approval_ANDROID,device_approval_IOS,device_approval_API,product_id_1,product_id_2,gender_female,gender_male,province_JAWA BARAT,province_DKI JAKARTA,province_JAWA TIMUR,province_JAWA TENGAH,province_BANTEN,province_SUMATERA UTARA,province_SULAWESI SELATAN,province_SUMATERA SELATAN,province_BALI,province_KALIMANTAN TIMUR,province_DAERAH ISTIMEWA YOGYAKARTA,province_LAMPUNG,province_SULAWESI UTARA,province_RIAU,province_SUMATERA BARAT,province_KEPULAUAN RIAU,province_KALIMANTAN BARAT,province_KALIMANTAN SELATAN,province_JAMBI,province_NANGGROE ACEH DARUSSALAM,province_KALIMANTAN TENGAH,province_NUSA TENGGARA BARAT,province_SULAWESI TENGAH,province_BENGKULU,province_NUSA TENGGARA TIMUR,province_KEPULAUAN BANGKA BELITUNG,province_SULAWESI TENGGARA,province_GORONTALO,province_PAPUA,province_KALIMANTAN UTARA,province_MALUKU,province_PAPUA BARAT,province_MALUKU UTARA,province_SULAWESI BARAT,province_ACEH,industry_involved_TRADING_INDUSTRY,industry_involved_OTHER,industry_involved_PABRIK,industry_involved_RETAIL,industry_involved_FINANCIAL,industry_involved_TRANSPORTASI_LOGISTIK,industry_involved_ROOM_BOARD,industry_involved_MEDICAL,industry_involved_EDUCATION,industry_involved_ARCHITECTURE,industry_involved_GOVERNMENT,industry_involved_INTERNET,industry_involved_E-COMMERCE,industry_involved_PKA,industry_involved_REAL_ESTAT,industry_involved_EXPERTS_TECHNOLOGY,industry_involved_,industry_involved_COMMERCE,marital_status_MARRIED,marital_status_SPINSTERHOOD,marital_status_DIVORCED,marital_status_BEREFT,marital_status_,education_SENIOR_HIGH_SCHOOL,education_REGULAR_COLLEGE_COURSE,education_ASSOCIATE_BACHELOR_3,education_ASSOCIATE_BACHELOR_1,education_JUNIOR_HIGH_SCHOOL,education_MASTER,education_ASSOCIATE_BACHELOR_2,education_PRIMARY_SCHOOL,education_,education_DOCTOR,cnt_loan30,cnt_loan60,cnt_loan90,cnt_loan180,cnt_loan360,cnt_loan,firstapply_curds,lastapply_curds,firstloan_curds,last_loan_curds,max_loan_period,max_loan_period30d,max_loan_period60d,max_loan_period90d,max_loan_period180d,max_loan_period360d,min_loan_period,min_loan_period30d,min_loan_period60d,min_loan_period90d,min_loan_period180d,min_loan_period360d,avg_loan_period,avg_loan_period30d,avg_loan_period60d,avg_loan_period90d,avg_loan_period180d,avg_loan_period360d,cnt_loan_period_is15,cnt_loan_period_is15_30d,cnt_loan_period_is15_60d,cnt_loan_period_is15_90d,cnt_loan_period_is15_180d,cnt_loan_period_is15_360d,cnt_loan_period_is8,cnt_loan_period_is8_30d,cnt_loan_period_is8_60d,cnt_loan_period_is8_90d,cnt_loan_period_is8_180d,cnt_loan_period_is8_360d,rate_loan_period_is15,rate_loan_period_is15_30d,rate_loan_period_is15_60d,rate_loan_period_is15_90d,rate_loan_period_is15_180d,rate_loan_period_is15_360d,rate_loan_period_is8,rate_loan_period_is8_30d,rate_loan_period_is8_60d,rate_loan_period_is8_90d,rate_loan_period_is8_180d,rate_loan_period_is8_360d,cnt_extend_times,cnt_extend_times_30d,cnt_extend_times_60d,cnt_extend_times_90d,cnt_extend_times_180d,cnt_extend_times_360d,max_extend_times,max_extend_times_30d,max_extend_times_60d,max_extend_times_90d,max_extend_times_180d,max_extend_times_360d,sum_extend_times,sum_extend_times_30d,sum_extend_times_60d,sum_extend_times_90d,sum_extend_times_180d,sum_extend_times_360d,avg_extend_times,avg_extend_times_30d,avg_extend_times_60d,avg_extend_times_90d,avg_extend_times_180d,avg_extend_times_360d,cnt_advance_paidoff,cnt_advance_paidoff30d,cnt_advance_paidoff60d,cnt_advance_paidoff90d,cnt_advance_paidoff180d,cnt_advance_paidoff360d,cnt_collection_paidoff,cnt_collection_paidoff30d,cnt_collection_paidoff60d,cnt_collection_paidoff90d,cnt_collection_paidoff180d,cnt_collection_paidoff360d,cnt_paidoff,cnt_paidoff30d,cnt_paidoff60d,cnt_paidoff90d,cnt_paidoff180d,cnt_paidoff360d,max_his_latedays,max_his_latedays_30d,max_his_latedays_60d,max_his_latedays_90d,max_his_latedays_180d,max_his_latedays_360d,min_his_latedays,min_his_latedays_30d,min_his_latedays_60d,

# 缺失值处理

In [11]:
df_all = load_data_from_pickle(file_path,'df_all.pkl')
df_all.fillna(-1,inplace = True) 
save_data_to_pickle(df_all,file_path,'all_var.pkl')
print(1)

1


In [12]:
var_dict['数据源'].value_counts()

app          222
app_tfidf    188
history      166
baseinfo      77
izi           27
app_freq       7
Name: 数据源, dtype: int64